In [2]:
# This notebook aims at fixing SR - Detection
# Train Yolo more with SR images

#%%
# Standard libraries imports
import importlib
import os
import glob
import random
import shutil
from tqdm.autonotebook import tqdm
import pybboxes as pbx


# Data manipulation libraries imports (for image processing)
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Torch libraries imports
import torch
from torch import nn
from torch.nn.functional import softmax
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import StepLR


# # Pytorch Lightning libraries imports
# import pytorch_lightning as pl
# from pytorch_lightning import Trainer, seed_everything
# from pytorch_lightning.loggers import TensorBoardLogger
# from pytorch_lightning.loggers.csv_logs import CSVLogger
# from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

# Torch utilities libraries imports
import torchmetrics
from torchsummary import summary
from torchmetrics.image import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
# from torchvision.models import resnet101
from torchvision import transforms

torch.cuda.empty_cache()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#%%
ToK1871_annotation = 'NomDataset/datasets/mono-domain-datasets/tale-of-kieu/1871/1871-annotation/annotation-mynom/'
ToK1871_image = 'NomDataset/datasets/mono-domain-datasets/tale-of-kieu/1871/1871-raw-images/'

ToK1902_annotation = 'NomDataset/datasets/mono-domain-datasets/tale-of-kieu/1902/1902-annotation/annotation-mynom/'
Tok1902_image = 'NomDataset/datasets/mono-domain-datasets/tale-of-kieu/1902/1902-raw-images/'

LVT_annotation = 'NomDataset/datasets/mono-domain-datasets/luc-van-tien/lvt-annotation/annotation-mynom/'
LVT_image = 'NomDataset/datasets/mono-domain-datasets/luc-van-tien/lvt-raw-images/'

C:\Users\Ponpon\AppData\Local\Temp\ipykernel_3696\1579317541.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Make folders
if not os.path.exists('yolov5'):
    raise Exception('yolov5 not found. Please clone the repository first')

if not os.path.exists('yolov5/data/yolov5-data-one-label-SR-nom'):
    os.makedirs('yolov5/data/yolov5-data-one-label-SR-nom/train/images', exist_ok=True)
    os.makedirs('yolov5/data/yolov5-data-one-label-SR-nom/train/labels', exist_ok=True)
    os.makedirs('yolov5/data/yolov5-data-one-label-SR-nom/val/images', exist_ok=True)
    os.makedirs('yolov5/data/yolov5-data-one-label-SR-nom/val/labels', exist_ok=True)
    os.makedirs('yolov5/data/yolov5-data-one-label-SR-nom/test/images', exist_ok=True)
    os.makedirs('yolov5/data/yolov5-data-one-label-SR-nom/test/labels', exist_ok=True)

# ToK 1871 for training, 1902 for validation, LVT for testing


In [3]:
folder_path = 'TempResources\SR_from_HR_ToK1871'
num_items = len(os.listdir(folder_path))
print(f"Number of items inside the folder: {num_items}")

folder_path = 'TempResources\SR_from_HR_ToK1902'
num_items = len(os.listdir(folder_path))
print(f"Number of items inside the folder: {num_items}")

folder_path = 'TempResources\SR_from_HR_LVT'
num_items = len(os.listdir(folder_path))
print(f"Number of items inside the folder: {num_items}")

Number of items inside the folder: 137
Number of items inside the folder: 164
Number of items inside the folder: 105


In [4]:
ToK1871_SR_file_path_temp = 'TempResources\SR_from_HR_ToK1871'
ToK1902_SR_file_path_temp = 'TempResources\SR_from_HR_ToK1902'
LVT_SR_file_path_temp = 'TempResources\SR_from_HR_LVT'

ToK1871_SR_file_path_yolo = 'yolov5/data/yolov5-data-one-label-SR-nom/train/images'
ToK1902_SR_file_path_yolo = 'yolov5/data/yolov5-data-one-label-SR-nom/val/images'
LVT_SR_file_path_yolo = 'yolov5/data/yolov5-data-one-label-SR-nom/test/images'

# Copy 1871 to train, 1902 to val, LVT to test
for file in tqdm(os.listdir(ToK1871_SR_file_path_temp)):
    if os.path.exists(os.path.join(ToK1871_SR_file_path_yolo, file)):
        continue
    shutil.copy(os.path.join(ToK1871_SR_file_path_temp, file), ToK1871_SR_file_path_yolo)

for file in tqdm(os.listdir(ToK1902_SR_file_path_temp)):
    if os.path.exists(os.path.join(ToK1902_SR_file_path_yolo, file)):
        continue
    shutil.copy(os.path.join(ToK1902_SR_file_path_temp, file), ToK1902_SR_file_path_yolo)

for file in tqdm(os.listdir(LVT_SR_file_path_temp)):
    if os.path.exists(os.path.join(LVT_SR_file_path_yolo, file)):
        continue
    shutil.copy(os.path.join(LVT_SR_file_path_temp, file), LVT_SR_file_path_yolo)

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/164 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

In [5]:
folder_path = 'yolov5/data/yolov5-data-one-label-SR-nom/train/images'
num_items = len(os.listdir(folder_path))
print(f"Number of items inside the folder: {num_items}")

folder_path = 'yolov5/data/yolov5-data-one-label-SR-nom/val/images'
num_items = len(os.listdir(folder_path))
print(f"Number of items inside the folder: {num_items}")

folder_path = 'yolov5/data/yolov5-data-one-label-SR-nom/test/images'
num_items = len(os.listdir(folder_path))
print(f"Number of items inside the folder: {num_items}")

Number of items inside the folder: 137
Number of items inside the folder: 164
Number of items inside the folder: 105


In [25]:
# Create labels file
SCALE = 4

ToK1871_label_file_path_yolo = 'yolov5/data/yolov5-data-one-label-SR-nom/train/labels'
ToK1902_label_file_path_yolo = 'yolov5/data/yolov5-data-one-label-SR-nom/val/labels'
LVT_label_file_path_yolo = 'yolov5/data/yolov5-data-one-label-SR-nom/test/labels'

for excel_file, image_file in tqdm(zip(os.listdir(ToK1871_annotation), os.listdir(ToK1871_SR_file_path_yolo))):
    df = pd.read_excel(os.path.join(ToK1871_annotation, excel_file))
    img = cv2.imread(os.path.join(ToK1871_SR_file_path_yolo, image_file))
    h, w, _ = img.shape

    bbox_list = []
    for index, row in df.iterrows():
        x_tl, y_tl, x_br, y_br = row['LEFT'], row['TOP'], row['RIGHT'], row['BOTTOM']
        
        x_tl *= SCALE
        y_tl *= SCALE
        x_br *= SCALE
        y_br *= SCALE


        # print(x_tl, y_tl, x_br, y_br, "img shape", w, h)
        # cv2.rectangle(img, (x_tl, y_tl), (x_br, y_br), (0, 255, 0), 2)

        bbox = pbx.convert_bbox((x_tl, y_tl, x_br, y_br), from_type='voc', to_type='yolo', image_size=(w, h))
        bbox_list.append(bbox)

    # plt.imshow(img)

    class_id = 0
    label_file_path = os.path.join(ToK1871_label_file_path_yolo, image_file.split('.')[0] + '.txt')
    with open(label_file_path, 'w') as f:
        for bbox in bbox_list:
            f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
    
    # print(f"Label file " + image_file.split('.')[0] + ".txt created")

#================================================================================================

for excel_file, image_file in tqdm(zip(os.listdir(ToK1902_annotation), os.listdir(ToK1902_SR_file_path_yolo))):
    df = pd.read_excel(os.path.join(ToK1902_annotation, excel_file))
    img = cv2.imread(os.path.join(ToK1902_SR_file_path_yolo, image_file))
    h, w, _ = img.shape

    bbox_list = []
    for index, row in df.iterrows():
        x_tl, y_tl, x_br, y_br = row['LEFT'], row['TOP'], row['RIGHT'], row['BOTTOM']
        
        x_tl *= SCALE
        y_tl *= SCALE
        x_br *= SCALE
        y_br *= SCALE


        # print(x_tl, y_tl, x_br, y_br, "img shape", w, h)
        # cv2.rectangle(img, (x_tl, y_tl), (x_br, y_br), (0, 255, 0), 2)

        bbox = pbx.convert_bbox((x_tl, y_tl, x_br, y_br), from_type='voc', to_type='yolo', image_size=(w, h))
        bbox_list.append(bbox)

    # plt.imshow(img)

    class_id = 0
    label_file_path = os.path.join(ToK1902_label_file_path_yolo, image_file.split('.')[0] + '.txt')
    with open(label_file_path, 'w') as f:
        for bbox in bbox_list:
            f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
    
    # print(f"Label file " + image_file.split('.')[0] + ".txt created")

#================================================================================================ 

for excel_file, image_file in tqdm(zip(os.listdir(LVT_annotation), os.listdir(LVT_SR_file_path_yolo))):
    df = pd.read_excel(os.path.join(LVT_annotation, excel_file))
    img = cv2.imread(os.path.join(LVT_SR_file_path_yolo, image_file))
    h, w, _ = img.shape

    bbox_list = []
    for index, row in df.iterrows():
        x_tl, y_tl, x_br, y_br = row['LEFT'], row['TOP'], row['RIGHT'], row['BOTTOM']
        
        x_tl *= SCALE
        y_tl *= SCALE
        x_br *= SCALE
        y_br *= SCALE


        # print(x_tl, y_tl, x_br, y_br, "img shape", w, h)
        # cv2.rectangle(img, (x_tl, y_tl), (x_br, y_br), (0, 255, 0), 2)

        bbox = pbx.convert_bbox((x_tl, y_tl, x_br, y_br), from_type='voc', to_type='yolo', image_size=(w, h))
        bbox_list.append(bbox)

    # plt.imshow(img)

    class_id = 0
    label_file_path = os.path.join(LVT_label_file_path_yolo, image_file.split('.')[0] + '.txt')
    with open(label_file_path, 'w') as f:
        for bbox in bbox_list:
            f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
    
    # print(f"Label file " + image_file.split('.')[0] + ".txt created")




0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [1]:
!wandb login b68403205f19bf63bece72cee92107c44a322afe


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Ponpon\.netrc


In [4]:
%cd yolov5
!python train.py --img 640 --batch 16 --epochs 30 --data "./data/yolov5-data-one-label-SR-nom.yaml" --weights "./weights/yolo_one_label.pt" 

[WinError 2] The system cannot find the file specified: 'yolov5'
d:\Documents\GitHub\Thesis\yolov5


C:\Users\Ponpon\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


^C


In [3]:
if not os.path.exists('yolov5/data/yolov5-data-one-label-mixed-nom'):
    os.makedirs('yolov5/data/yolov5-data-one-label-mixed-nom/train/images', exist_ok=True)
    os.makedirs('yolov5/data/yolov5-data-one-label-mixed-nom/train/labels', exist_ok=True)
    os.makedirs('yolov5/data/yolov5-data-one-label-mixed-nom/val/images', exist_ok=True)
    os.makedirs('yolov5/data/yolov5-data-one-label-mixed-nom/val/labels', exist_ok=True)
    os.makedirs('yolov5/data/yolov5-data-one-label-mixed-nom/test/images', exist_ok=True)
    os.makedirs('yolov5/data/yolov5-data-one-label-mixed-nom/test/labels', exist_ok=True)

In [6]:
ToK1871_SR_file_path_temp = 'TempResources\SR_from_HR_ToK1871'
ToK1902_SR_file_path_temp = 'TempResources\SR_from_HR_ToK1902'
LVT_SR_file_path_temp = 'TempResources\SR_from_HR_LVT'

ToK1871_SR_file_path_yolo = 'yolov5/data/yolov5-data-one-label-mixed-nom/train/images'
ToK1902_SR_file_path_yolo = 'yolov5/data/yolov5-data-one-label-mixed-nom/val/images'
LVT_SR_file_path_yolo = 'yolov5/data/yolov5-data-one-label-mixed-nom/test/images'

# Copy 1871 to train, 1902 to val, LVT to test
for file in tqdm(os.listdir(ToK1871_SR_file_path_temp)):
    if os.path.exists(os.path.join(ToK1871_SR_file_path_yolo, file)):
        continue
    shutil.copy(os.path.join(ToK1871_SR_file_path_temp, file), ToK1871_SR_file_path_yolo)
for file in tqdm(os.listdir(ToK1871_image)):
    if os.path.exists(os.path.join(ToK1871_SR_file_path_yolo, file)):
        continue
    shutil.copy(os.path.join(ToK1871_image, file), ToK1871_SR_file_path_yolo)


for file in tqdm(os.listdir(ToK1902_SR_file_path_temp)):
    if os.path.exists(os.path.join(ToK1902_SR_file_path_yolo, file)):
        continue
    shutil.copy(os.path.join(ToK1902_SR_file_path_temp, file), ToK1902_SR_file_path_yolo)
for file in tqdm(os.listdir(Tok1902_image)):
    if os.path.exists(os.path.join(ToK1902_SR_file_path_yolo, file)):
        continue
    shutil.copy(os.path.join(Tok1902_image, file), ToK1902_SR_file_path_yolo)


for file in tqdm(os.listdir(LVT_SR_file_path_temp)):
    if os.path.exists(os.path.join(LVT_SR_file_path_yolo, file)):
        continue
    shutil.copy(os.path.join(LVT_SR_file_path_temp, file), LVT_SR_file_path_yolo)
for file in tqdm(os.listdir(LVT_image)):
    if os.path.exists(os.path.join(LVT_SR_file_path_yolo, file)):
        continue
    shutil.copy(os.path.join(LVT_image, file), LVT_SR_file_path_yolo)



  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/164 [00:00<?, ?it/s]

  0%|          | 0/164 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

In [8]:
folder_path = 'yolov5/data/yolov5-data-one-label-mixed-nom/train/images'
num_items = len(os.listdir(folder_path))
print(f"Number of items inside the folder: {num_items}")

folder_path = 'yolov5/data/yolov5-data-one-label-mixed-nom/val/images'
num_items = len(os.listdir(folder_path))
print(f"Number of items inside the folder: {num_items}")

folder_path = 'yolov5/data/yolov5-data-one-label-mixed-nom/test/images'
num_items = len(os.listdir(folder_path))
print(f"Number of items inside the folder: {num_items}")

Number of items inside the folder: 274
Number of items inside the folder: 328
Number of items inside the folder: 210


In [10]:
# Create labels file
SCALE = 4

ToK1871_label_file_path_yolo = 'yolov5/data/yolov5-data-one-label-mixed-nom/train/labels'
ToK1902_label_file_path_yolo = 'yolov5/data/yolov5-data-one-label-mixed-nom/val/labels'
LVT_label_file_path_yolo = 'yolov5/data/yolov5-data-one-label-mixed-nom/test/labels'

for excel_file, image_file in tqdm(zip(os.listdir(ToK1871_annotation), os.listdir(ToK1871_SR_file_path_yolo))):
    df = pd.read_excel(os.path.join(ToK1871_annotation, excel_file))
    img = cv2.imread(os.path.join(ToK1871_SR_file_path_yolo, image_file))
    h, w, _ = img.shape

    bbox_list = []
    for index, row in df.iterrows():
        x_tl, y_tl, x_br, y_br = row['LEFT'], row['TOP'], row['RIGHT'], row['BOTTOM']
        
        x_tl *= SCALE
        y_tl *= SCALE
        x_br *= SCALE
        y_br *= SCALE


        # print(x_tl, y_tl, x_br, y_br, "img shape", w, h)
        # cv2.rectangle(img, (x_tl, y_tl), (x_br, y_br), (0, 255, 0), 2)

        bbox = pbx.convert_bbox((x_tl, y_tl, x_br, y_br), from_type='voc', to_type='yolo', image_size=(w, h))
        bbox_list.append(bbox)

    # plt.imshow(img)

    class_id = 0
    label_file_path = os.path.join(ToK1871_label_file_path_yolo, image_file.split('.')[0] + '.txt')
    with open(label_file_path, 'w') as f:
        for bbox in bbox_list:
            f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
    
    # print(f"Label file " + image_file.split('.')[0] + ".txt created")
            

#================================================================================================

for excel_file, image_file in tqdm(zip(os.listdir(ToK1902_annotation), os.listdir(ToK1902_SR_file_path_yolo))):
    df = pd.read_excel(os.path.join(ToK1902_annotation, excel_file))
    img = cv2.imread(os.path.join(ToK1902_SR_file_path_yolo, image_file))
    h, w, _ = img.shape

    bbox_list = []
    for index, row in df.iterrows():
        x_tl, y_tl, x_br, y_br = row['LEFT'], row['TOP'], row['RIGHT'], row['BOTTOM']
        
        x_tl *= SCALE
        y_tl *= SCALE
        x_br *= SCALE
        y_br *= SCALE


        # print(x_tl, y_tl, x_br, y_br, "img shape", w, h)
        # cv2.rectangle(img, (x_tl, y_tl), (x_br, y_br), (0, 255, 0), 2)

        bbox = pbx.convert_bbox((x_tl, y_tl, x_br, y_br), from_type='voc', to_type='yolo', image_size=(w, h))
        bbox_list.append(bbox)

    # plt.imshow(img)

    class_id = 0
    label_file_path = os.path.join(ToK1902_label_file_path_yolo, image_file.split('.')[0] + '.txt')
    with open(label_file_path, 'w') as f:
        for bbox in bbox_list:
            f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
    
    # print(f"Label file " + image_file.split('.')[0] + ".txt created")

#================================================================================================ 

for excel_file, image_file in tqdm(zip(os.listdir(LVT_annotation), os.listdir(LVT_SR_file_path_yolo))):
    df = pd.read_excel(os.path.join(LVT_annotation, excel_file))
    img = cv2.imread(os.path.join(LVT_SR_file_path_yolo, image_file))
    h, w, _ = img.shape

    bbox_list = []
    for index, row in df.iterrows():
        x_tl, y_tl, x_br, y_br = row['LEFT'], row['TOP'], row['RIGHT'], row['BOTTOM']
        
        x_tl *= SCALE
        y_tl *= SCALE
        x_br *= SCALE
        y_br *= SCALE


        # print(x_tl, y_tl, x_br, y_br, "img shape", w, h)
        # cv2.rectangle(img, (x_tl, y_tl), (x_br, y_br), (0, 255, 0), 2)

        bbox = pbx.convert_bbox((x_tl, y_tl, x_br, y_br), from_type='voc', to_type='yolo', image_size=(w, h))
        bbox_list.append(bbox)

    # plt.imshow(img)

    class_id = 0
    label_file_path = os.path.join(LVT_label_file_path_yolo, image_file.split('.')[0] + '.txt')
    with open(label_file_path, 'w') as f:
        for bbox in bbox_list:
            f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
    
    # print(f"Label file " + image_file.split('.')[0] + ".txt created")

#====================================================================================================
#====================================================================================================
#====================================================================================================
SCALE = 1

for excel_file, image_file in tqdm(zip(os.listdir(ToK1871_annotation), os.listdir(ToK1871_image))):
    df = pd.read_excel(os.path.join(ToK1871_annotation, excel_file))
    img = cv2.imread(os.path.join(ToK1871_image, image_file))
    h, w, _ = img.shape

    bbox_list = []
    for index, row in df.iterrows():
        x_tl, y_tl, x_br, y_br = row['LEFT'], row['TOP'], row['RIGHT'], row['BOTTOM']
        
        x_tl *= SCALE
        y_tl *= SCALE
        x_br *= SCALE
        y_br *= SCALE


        # print(x_tl, y_tl, x_br, y_br, "img shape", w, h)
        # cv2.rectangle(img, (x_tl, y_tl), (x_br, y_br), (0, 255, 0), 2)

        bbox = pbx.convert_bbox((x_tl, y_tl, x_br, y_br), from_type='voc', to_type='yolo', image_size=(w, h))
        bbox_list.append(bbox)

    # plt.imshow(img)

    class_id = 0
    label_file_path = os.path.join(ToK1871_label_file_path_yolo, image_file.split('.')[0] + '.txt')
    with open(label_file_path, 'w') as f:
        for bbox in bbox_list:
            f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
    
    # print(f"Label file " + image_file.split('.')[0] + ".txt created")
            

#================================================================================================

for excel_file, image_file in tqdm(zip(os.listdir(ToK1902_annotation), os.listdir(Tok1902_image))):
    df = pd.read_excel(os.path.join(ToK1902_annotation, excel_file))
    img = cv2.imread(os.path.join(Tok1902_image, image_file))
    h, w, _ = img.shape

    bbox_list = []
    for index, row in df.iterrows():
        x_tl, y_tl, x_br, y_br = row['LEFT'], row['TOP'], row['RIGHT'], row['BOTTOM']
        
        x_tl *= SCALE
        y_tl *= SCALE
        x_br *= SCALE
        y_br *= SCALE


        # print(x_tl, y_tl, x_br, y_br, "img shape", w, h)
        # cv2.rectangle(img, (x_tl, y_tl), (x_br, y_br), (0, 255, 0), 2)

        bbox = pbx.convert_bbox((x_tl, y_tl, x_br, y_br), from_type='voc', to_type='yolo', image_size=(w, h))
        bbox_list.append(bbox)

    # plt.imshow(img)

    class_id = 0
    label_file_path = os.path.join(ToK1902_label_file_path_yolo, image_file.split('.')[0] + '.txt')
    with open(label_file_path, 'w') as f:
        for bbox in bbox_list:
            f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
    
    # print(f"Label file " + image_file.split('.')[0] + ".txt created")

#================================================================================================ 

for excel_file, image_file in tqdm(zip(os.listdir(LVT_annotation), os.listdir(LVT_image))):
    df = pd.read_excel(os.path.join(LVT_annotation, excel_file))
    img = cv2.imread(os.path.join(LVT_image, image_file))
    h, w, _ = img.shape

    bbox_list = []
    for index, row in df.iterrows():
        x_tl, y_tl, x_br, y_br = row['LEFT'], row['TOP'], row['RIGHT'], row['BOTTOM']
        
        x_tl *= SCALE
        y_tl *= SCALE
        x_br *= SCALE
        y_br *= SCALE


        # print(x_tl, y_tl, x_br, y_br, "img shape", w, h)
        # cv2.rectangle(img, (x_tl, y_tl), (x_br, y_br), (0, 255, 0), 2)

        bbox = pbx.convert_bbox((x_tl, y_tl, x_br, y_br), from_type='voc', to_type='yolo', image_size=(w, h))
        bbox_list.append(bbox)

    # plt.imshow(img)

    class_id = 0
    label_file_path = os.path.join(LVT_label_file_path_yolo, image_file.split('.')[0] + '.txt')
    with open(label_file_path, 'w') as f:
        for bbox in bbox_list:
            f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
    
    # print(f"Label file " + image_file.split('.')[0] + ".txt created")


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]